In [1]:
%autosave 0

Autosave disabled


## Preprocessing

In [2]:
user = 'PART2'
metal = 'ZN'
model_name = metal

In [3]:
import sys

print ("Initializing global variables...", end=' ')
sys.stdout.flush()

# shared result file
output_file = './logs/results.txt'
hist_path = model_path = fig_path = './logs/'
dict_path = './dictionaries/'

print ("Done")
# print ("  Filepath set to [" + user + "]'s directory")
print ("  Filepath set to ./logs/")


##################################################

print ("Importing modules...", end=' ')
import modules
print ("Done")

##################################################

print ("Reading data from disk...", end=' ')
sys.stdout.flush()

import numpy as np
import pandas as pd

df = pd.read_parquet('./datasets/Metal_all_20180601_predicted.parquet')


# Extract the metal
df_metal = df.loc[df['metalPrediction'] == metal]
# df_zn_single = df_zn.loc[df_zn['interactingChains'] == 1]

seqs = np.array(df_metal.sequence)
target = np.array(df_metal.fingerprint)
cluster_numbers = np.array(df_metal.clusterNumber90)

# del df,df_zn,df_zn_single


print ("Done [" + metal + "]")

##################################################

# print ("Removing 'U','X', NaN...", end=' ')
# sys.stdout.flush()
# # Remove seqs containing 'U' and 'X'
# rows_to_delete = []
# for i in range(seqs.shape[0]):
#     if 'X' in seqs[i] or 'U' in seqs[i] or np.isnan(cluster_numbers[i]):
#         rows_to_delete.append(i) 
#         print ("Removing", i)



# seqs = np.delete(seqs, rows_to_delete, 0)
# target = np.delete(target, rows_to_delete)
# cluster_numbers = np.delete(cluster_numbers, rows_to_delete)
# print ("Done")

##################################################

print ("Loading dictionaries...", end=' ')
sys.stdout.flush()
import json

# # ProtVec
# seqs_dict_w2v = {}
# with open(dict_path + "seq_n_gram_to_vec_dict_w_UX", 'r') as fp:
#         seqs_dict_w2v = json.load(fp)

# # One-hot
# seqs_dict_onehot = {}
# with open(dict_path + "seqs_dict_onehot", 'r') as fp:
#         seqs_dict_onehot = json.load(fp)

# FOFE
vocab_dic_fofe = {}
with open(dict_path + "vocab_dict_fofe", 'r') as fp:
        vocab_dic_fofe = json.load(fp)
            
            
# # property
# # blosum62
# from proteinSequenceEncoder import property_encoder, blosum62_encoder 
# AMINO_ACIDS21 = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', \
#                  'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y']
# seqs_property = {}
# seqs_blosum62 = {}
# seqs_onehot_blosum = {}

# for aa in AMINO_ACIDS21:
#     seqs_property[aa] = property_encoder(aa)
#     seqs_blosum62[aa] = blosum62_encoder(aa)
#     if aa != 'X' and aa !='U':
#         seqs_onehot_blosum[aa] = seqs_dict_onehot[aa] + blosum62_encoder(aa)[0]


print ("Done")

##################################################

print ("Performing cross validation split...", end=' ')
ratio = 0.9
split = int(ratio*len(seqs))
train_seqs, val_seqs = seqs[:split], seqs[split:]
train_label, val_label = target[:split], target[split:]
train_cluster, val_cluster = cluster_numbers[:split], cluster_numbers[split:]
print ("Done")
print ("  Ratio :", ratio)
print ("  Train_range :", 0, "-", split-1)
print ("  Val_range :", split, "-", len(seqs)-1)

Initializing global variables... Done
  Filepath set to ./logs/
Importing modules... 

Using TensorFlow backend.


Done
Reading data from disk... Done [ZN]
Loading dictionaries... Done
Performing cross validation split... Done
  Ratio : 0.9
  Train_range : 0 - 17279
  Val_range : 17280 - 19199


In [4]:
train_args = {'sequences': train_seqs,
              'labels': train_label,
              'translator': vocab_dic_fofe}
val_args = {'sequences': val_seqs,
            'labels': val_label,
            'translator': vocab_dic_fofe}
common_args = {'batch_size': 100,
               'input_shape': (800,),
               'label_shape': (706, ),
               'shuffle': True}

train_gen = modules.FOFEGenerator(**train_args, **common_args)
val_gen = modules.FOFEGenerator(**val_args, **common_args)

In [5]:
# ProtVec:100, One-hot:20, blosum62:20, property:7
dimension = 800
cutoff = 706

import tensorflow as tf
import time
import matplotlib.pyplot as plt
% matplotlib inline
np.random.seed(2017) 
from keras.models import Sequential, Model
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D, AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, Reshape, Embedding, Input
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import SGD
import numpy as np
import keras
from keras.models import Model, load_model
from keras.optimizers import Adam, SGD, RMSprop
# Visualization
from keras.utils import plot_model

input_shape = (dimension,)

input_0 = Input(shape=input_shape, dtype='float32')
input_0_reshape = Reshape((1,dimension,1), input_shape=(dimension,))(input_0)
conv2d_3 = Convolution2D(2, 1, 3, border_mode='same')(input_0_reshape)
conv2d_5 = Convolution2D(2, 1, 5, border_mode='same')(input_0_reshape)
conv2d_7 = Convolution2D(2, 1, 7, border_mode='same')(input_0_reshape)

x = keras.layers.concatenate([conv2d_3,conv2d_5,conv2d_7])
x = Activation('relu')(x)
x = Flatten()(x)
x = Dense(cutoff, activation='relu')(x)
output_0 = Dense(cutoff, activation='softmax')(x)
#output_0_reshape = Reshape((cutoff,1), input_shape=(cutoff,))(output_0)

#model = Model(inputs=input_0, outputs=output_0_reshape)
model = Model(inputs=input_0, outputs=output_0)                              
# end of the MODEL

sgd = SGD(lr = 0.01, momentum = 0.9, decay = 0, nesterov = False)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()

C:\Users\Tian\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 3), padding="same")`
C:\Users\Tian\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 5), padding="same")`
C:\Users\Tian\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 7), padding="same")`


- <font color=blue>Threshold: mean+2.33*std</font>

In [6]:
factor = 2.33
def threshold_func(y_in):
    factor = 2.33
    y_out = np.zeros_like(y_in)
    for i in range(y_in.shape[0]):
        th= np.mean(y_in[i]) + factor * np.std(y_in[i])
        y_out[i] = (y_in[i] > th)
    return y_out

- <font color=blue>Metric: F1 score</font>

In [7]:
cb = modules.F1_history(threshold_func)

model_args = {'model': model, 
              'generators': [train_gen, val_gen], 
              'callbacks': [cb], 
              'post_train_args': {'user': user, 
                                  'model': model_name, 
                                  'result': output_file, 
                                  'fig_path': fig_path, 
                                  'optimizer': str(type(model.optimizer)).replace('<class \'keras.optimizers.', '').replace('\'>', ''), 
                                  'optimizer_config' : model.optimizer.get_config(), 
                                  'loss': model.loss, 
                                  'factor': factor}}

trainer = modules.Trainer(**model_args)

Callback initialized.
Assigning validation generator... Done
Matching input shape... Done
Matching output shape... Done
Trainer initialized.


In [ ]:
import warnings; 
warnings.simplefilter('ignore')
trainer.start(epoch=10)

Epoch 1/10


In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open(metal + ".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(metal + ".h5")
print("Saved model to disk")

In [ ]:
# later...
# load json and create model
json_file = open(metal + ".json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(metal + ".h5")
print("Loaded model from disk")